در پیشبینی با استفاده از محتوا ما باید با استفاده از داده ها و امتیازهایی که داریم، ژانرهای مورد علاقه کاربر و امتیازدهی او به فیلم ها

 برای هر فیلم یک امتیاز وزن دهی شده اختصاص بدهیم و فیلم هایی که بیشترین امتیاز را داشتند را به کاربر پیشنهاد بدهیم

In [1]:
import numpy as np
import pandas as pd

 دیتافریم های مورد نیاز را میسازیم و ستون تایم استمپ را دراپ میکنیم چون ممکن است با استفاده از آن پیشبینی ما با خطا مواجه شود
 
 (چون مدت زمان روی صفحه ماندن میتواند دلایل مختلف و گاهی بی ربط داشته باشد)

In [2]:
rating_df = pd.read_csv("ratings.csv")
movie_df = pd.read_csv("movies.csv")

rating_df = rating_df.drop("timestamp", axis=1)
rating_df.head(5)
movie_df.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


با استفاده از ریجکس سال را از عنوان هر فیلم جدا میکنیم و داخل یک ستون مجزا قرار میدهیم و ستون عنوان فیلم ها را هم سامان میدهیم.

 ژانر فیلم ها را از فرم اولیشان خارج میکنیم و به صورت لیست ذخیره میکنیم

In [3]:
titles = movie_df["title"]
movie_df["year"] = movie_df["title"].str.extract('(\(\d\d\d\d\))',expand=False)
movie_df["year"] = movie_df["year"].str.extract('(\d\d\d\d)',expand=False)
movie_df["title"] = movie_df["title"].str.replace('(\(\d\d\d\d\))', '')
movie_df["title"] = movie_df["title"].apply(lambda x : x.strip())
movie_df["genres"] = movie_df["genres"].str.split("|")
movie_df.head()

,movieId,title,genres,year
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II (1995),[Comedy],1995


از دیتافریم قبلی یک نمونه جدید میسازیم، با استفاده از حلقه هر فیلمی که ژانری را داشته باشد را با 1 نمایش میدهیم.
 
 بعد در خارج از حلقه جاهایی که با 1 پر نشدند و مقدار عددی ندارند را با 0 پر میکنیم

In [4]:
new_movie_df = movie_df.copy()

for index, row in new_movie_df.iterrows():
    for genre in row["genres"]:
        new_movie_df.at[index, genre] = 1

new_movie_df = new_movie_df.fillna(0)

new_movie_df['title'] = new_movie_df['title'].str.replace("\"", "")

new_movie_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


یک یوزر نمونه میسازیم! ودیتافریم آن را تشکیل میدهیم

In [5]:
27,34,5.0,962685351
27,48,4.0,962685318
27,60,3.0,962685387
27,62,5.0,962684909
27,126,2.0,962685762

user = [
            {'title':'Babe (1995)', 'rating':5},
            {'title':'Pocahontas (1995)', 'rating':4},
            {'title':'Indian in the Cupboard, The (1995)', 'rating':3},
            {'title':"Mr. Holland's Opus (1995)", 'rating':5},
            {'title':'NeverEnding Story III, The (1994)', 'rating':2}
         ] 
user_df = pd.DataFrame(user)
user_df

,title,rating
0,Babe (1995),5
1,Pocahontas (1995),4
2,"Indian in the Cupboard, The (1995)",3
3,Mr. Holland's Opus (1995),5
4,"NeverEnding Story III, The (1994)",2


در اینجا ما به دنبال فیلم هایی هستیم که یوزر ساختگیمون آنها را پسند کرده،

 این فیلم ها را در دیتافریمی که برای فیلم ها ساختیم پیدا میکنیم و یک دیتافریم جدید میسازیم که حاوی آیدی، عنوان و امتیاز فیلم های کاربر ما میباشد

In [6]:
inputId = movie_df[movie_df['title'].isin(user_df['title'].tolist())]
inputMovies = pd.merge(inputId, user_df)
inputMovies = inputMovies.drop('genres', axis=1).drop('year', axis=1)
inputMovies

,movieId,title,rating
0,34,Babe (1995),5
1,48,Pocahontas (1995),4
2,60,"Indian in the Cupboard, The (1995)",3
3,62,Mr. Holland's Opus (1995),5
4,126,"NeverEnding Story III, The (1994)",2


یک دیتافریم هم به همراه ژانرهای فیلم هایی که از کاربر داریم میسازیم

In [7]:
userMovies = new_movie_df[new_movie_df["movieId"].isin(inputMovies["movieId"].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
32,34,Babe (1995),"[Children, Drama]",1995,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,48,Pocahontas (1995),"[Animation, Children, Drama, Musical, Romance]",1995,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
53,60,"Indian in the Cupboard, The (1995)","[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,62,Mr. Holland's Opus (1995),[Drama],1995,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,126,"NeverEnding Story III, The (1994)","[Adventure, Children, Fantasy]",1994,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


یک دیتافریم صرفا از ژانرها میسازیم و ژانرهای مورد علاقه کاربر را مشاهده میکنیم

In [8]:
userMovies = userMovies.reset_index(drop=True)
userGenreTable = userMovies.drop("movieId", axis=1).drop("title", axis=1).drop("genres", axis=1).drop("year", axis=1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
inputMovies["rating"]

0    5
1    4
2    3
3    5
4    2
Name: rating, dtype: int64

ماتریسی که استاد در درسا گفتن را میسازیم و به هر ژانر یک عدد اختصاص میدهیم که نشان دهنده میزان علاقه کاربر به آن ژانر است

In [10]:
userProfile = userGenreTable.transpose().dot(inputMovies["rating"])
userProfile

Adventure              5.0
Animation              4.0
Children              14.0
Comedy                 0.0
Fantasy                5.0
Romance                4.0
Drama                 14.0
Action                 0.0
Crime                  0.0
Thriller               0.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 0.0
War                    0.0
Musical                4.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [11]:
genreTable = new_movie_df.set_index(new_movie_df['movieId'])
genreTable = genreTable.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year', axis=1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
genreTable.shape

(9742, 20)

حالا به امتیازهایی که به ژانرها دادیم اصطلاحا وزن میدهیم

In [13]:
recTable_df = ((genreTable * userProfile).sum(axis=1)) / (userProfile.sum())
recTable_df.head()

movieId
1    0.56
2    0.48
3    0.08
4    0.36
5    0.00
dtype: float64

دیتافریم فیلم های پیشنهادیمون به کاربر رو مرتب میکنیم تا بالاترین امتیازات وزن دهی شده را پیدا کنیم

In [14]:
recTable_df = recTable_df.sort_values(ascending=False)
recTable_df.head()

movieId
1907      0.90
134853    0.84
2800      0.84
26662     0.84
2138      0.84
dtype: float64

حالا با استاده از آیدی فیلم هایی که پیدا کردیم 5 فیلم برتر را از دیتافریم فیلم ها پیدا میکنیم

In [15]:
result = movie_df.loc[movie_df['movieId'].isin(recTable_df.head(20).keys())]
resMovies = result[:5]
resMovies

,movieId,title,genres,year
44,48,Pocahontas (1995),"[Animation, Children, Drama, Musical, Romance]",1995
322,364,"Lion King, The (1994)","[Adventure, Animation, Children, Drama, Musica...",1994
511,594,Snow White and the Seven Dwarfs (1937),"[Animation, Children, Drama, Fantasy, Musical]",1937
618,783,"Hunchback of Notre Dame, The (1996)","[Animation, Children, Drama, Musical, Romance]",1996
1273,1688,Anastasia (1997),"[Adventure, Animation, Children, Drama, Musical]",1997


پیشنهادات را نمایش میدهیم

In [16]:
Movies = resMovies["title"].tolist()

print("I recommend to you these movies: \n")
for movie in Movies:
    print("    " + movie, end="\n\n")

I recommend to you these movies: 

    Pocahontas (1995)

    Lion King, The (1994)

    Snow White and the Seven Dwarfs (1937)

    Hunchback of Notre Dame, The (1996)

    Anastasia (1997)

